#Invoke AI Notebook

Works on the free tier: Generating images with the SDXL base model and refiner. Adding SDXL models in diffusers format from HuggingFace.

Works, but only with Colab Pro: Adding custom checkpoints and LoRAs.

Step 1

In [ ]:
#@markdown # Installing InvokeAI

#@markdown Use Google Drive to store models (uses about 7 GB). Uncheck this if you don't have enough space in your Drive.
useGoogleDrive = True #@param {type:"boolean"}

googleDriveModelsFolder = '/stablemodels' #@param {type:"string"}

#@markdown This step usually takes about 5 minutes.

# @markdown You can ignore the message about restarting the runtime. Additionally, warnings about incompatible packages can be ignored as well.
import os
import subprocess
from google.colab import drive
if useGoogleDrive:
  drive.mount('/content/drive')
  if not googleDriveModelsFolder.startswith('/'):
    googleDriveModelsFolder = '/' + googleDriveModelsFolder
  modelsPath = "/content/drive/MyDrive"+googleDriveModelsFolder
  if not modelsPath.endswith("/"):
   modelsPath = modelsPath + "/"

env = os.environ.copy()

!pip install 'InvokeAI[xformers]' --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117

# Replace with this line to install a version that uses less RAM and won't crash
#!pip install 'InvokeAI[xformers]==3.1.1' --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117




Step 2

In [ ]:
#@markdown # Configuration and downloading default models

!mkdir /content/invokeai
!mkdir /content/invokeai/configs

#@markdown Download only the default model in initial configuration.
#@markdown Checking this prevents running out of space in Colab.

defaultOnly = True #@param {type:"boolean"}
skipWeights = True #@param {type:"boolean"}
noFullPrecision = True  #@param {type:"boolean"}
#@markdown This step usually takes about 2 minutes with only the default model and no weights.

#@markdown You can ignore "File exists" warnings in the output.

cmd = 'invokeai-configure --root_dir /content/invokeai --yes'

if defaultOnly:
  cmd += ' --default_only'

if skipWeights:
  cmd += ' --skip-sd-weights'

if noFullPrecision:
  cmd += ' --no-full-precision'

get_ipython().system(cmd)

import fileinput
import os
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

if noFullPrecision:
  model_install_file = find('model_install_backend.py', '/usr/local/lib')
  print('modifying file ' + model_install_file)
  for line in fileinput.input(model_install_file, inplace=True):
    if ('precision = torch_dtype(choose_torch_device())' in line):
       line = line.replace('torch_dtype(choose_torch_device())', 'torch.float16')
    print(line, end='')


Step 3 - Adding the SDXL Base Model - Optional, but at least one model is required

In [ ]:
#@markdown # Adding the SDXL Base Model

#@markdown Installing SDXL base took about 10 minutes initially, but it's finished instantly
#@markdown in subsequent runs if Google Drive is enabled. You can execute the first run in a runtime
#@markdown without a GPU to save the model to Google Drive without spending GPU time.

#@markdown This will delete anything in the folder /content/invokeai/models/sdxl

import os.path
from os import path

# Install the SDXL base model
def installSdxl():
  installCmd = 'invokeai-model-install --add "stabilityai/stable-diffusion-xl-base-1.0" --root_dir /content/invokeai'
  get_ipython().system(installCmd)

sdxlBaseSubfolderName = '/stable-diffusion-xl-base-1-0'

if useGoogleDrive:
  alreadyInstalled = True

  driveSdxlMainFolder = modelsPath + 'sdxl/main'
  if not path.exists(driveSdxlMainFolder):
    os.makedirs(driveSdxlMainFolder, exist_ok=True)
    alreadyInstalled = False

  localModelsSdxlFolder = '/content/invokeai/models/sdxl/'
  localSdxlMainFolder = localModelsSdxlFolder + 'main'

  get_ipython().system('rm -rf ' + localModelsSdxlFolder)
  get_ipython().system('rmdir ' + localModelsSdxlFolder)

  if not alreadyInstalled:
    if not path.exists(localModelsSdxlFolder):
      os.makedirs(localModelsSdxlFolder, exist_ok=True)
    get_ipython().system('ln -s '+driveSdxlMainFolder+' '+localModelsSdxlFolder)
    installSdxl()
    get_ipython().system('unlink '+localSdxlMainFolder)
    if not path.exists(localSdxlMainFolder):
      os.makedirs(localSdxlMainFolder, exist_ok=True)
    get_ipython().system('ln -s '+driveSdxlMainFolder + sdxlBaseSubfolderName+' '+ localSdxlMainFolder+ sdxlBaseSubfolderName)
  else:
    if not path.exists(localSdxlMainFolder):
      os.makedirs(localSdxlMainFolder, exist_ok=True)
    get_ipython().system('ln -s '+driveSdxlMainFolder + sdxlBaseSubfolderName+' '+ localSdxlMainFolder+ sdxlBaseSubfolderName)
    updateModelsYaml = True
    with open('/content/invokeai/configs/models.yaml') as f:
      if 'stable-diffusion-xl-base-1-0' in f.read():
        updateModelsYaml = False
    if updateModelsYaml:
      with open('/content/invokeai/configs/models.yaml', 'a') as file:
        lines = [
          'sdxl/main/stable-diffusion-xl-base-1-0:\n',
          '  path: sdxl/main/stable-diffusion-xl-base-1-0\n',
          '  description: Stable Diffusion XL base model (12 GB)\n',
          '  variant: normal\n',
          '  format: diffusers\n'
        ]
        file.writelines(lines)
else:
  installSdxl()


Step 4: Download the refiner model and vae - Optional

In [ ]:
#@markdown Adding the refiner and vae. This one took about 14 minutes.
#@markdown Skip this step if you don't need these models.
!invokeai-model-install --add "stabilityai/stable-diffusion-xl-refiner-1.0" --root_dir /content/invokeai --yes
!invokeai-model-install --add "madebyollin/sdxl-vae-fp16-fix" --root_dir /content/invokeai --yes

Step 5: Saving outputs to a Drive folder - Optional

In [ ]:
# Linking output images to Google Drive
outputDrivePath = '/content/drive/MyDrive/images/invoke-outputs' #@param {type:"string"}
# Full path to the output folder on Google Drive

saveDatabase = True #@param {type:"boolean"}
from os import path

from google.colab import drive
import os
from os import path
drive.mount('/content/drive')

if not outputDrivePath.endswith('/'):
  outputDrivePath = outputDrivePath + '/'
imagesDrivePath = outputDrivePath + 'images'
databaseDrivePath = outputDrivePath + 'databases'
if not path.exists(imagesDrivePath):
  os.makedirs(imagesDrivePath, exist_ok=True)


outputsLocalPath = '/content/invokeai/outputs'
imagesLocalPath = '/content/invokeai/outputs/images'

if not path.exists(outputsLocalPath):
  os.makedirs(outputsLocalPath, exist_ok=True)

import datetime

if path.exists(imagesLocalPath):
    cmd = f'mv {imagesLocalPath} {imagesLocalPath}-backup{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
    get_ipython().system(cmd)

cmd = f'ln -s {imagesDrivePath} {outputsLocalPath}'
get_ipython().system(cmd)

# Linking the database
if saveDatabase:
  if not path.exists(databaseDrivePath):
    os.makedirs(databaseDrivePath, exist_ok=True)

  databaseLocalPath = '/content/invokeai/databases'

  cmd = f'mv {databaseLocalPath} {databaseLocalPath}-backup{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
  get_ipython().system(cmd)

  cmd = f'ln -s {databaseDrivePath} /content/invokeai'
  get_ipython().system(cmd)


Step 6: Load any SDXL models in diffusers format from Drive - Optional

In [ ]:
# Adding custom SDXL models in diffusers format from Goole Drive
googleDriveModelFolder = '/content/drive/MyDrive/path-to-the-model' #@param {type:"string"}
#@markdown - Full path to the model folder on Google Drive

#@markdown This can also be done from the Model Manager in the Web UI.

updateModelsYaml = True
with open('/content/invokeai/configs/models.yaml') as f:
      if googleDriveModelFolder in f.read():
        updateModelsYaml = False
if updateModelsYaml:
      with open('/content/invokeai/configs/models.yaml', 'a') as file:
        folders = googleDriveModelFolder.split('/');
        modelname = folders[len(folders)-1]
        print(modelname)
        lines = [
          'sdxl/main/' + modelname + ':\n',
          '  path: ' + googleDriveModelFolder + '\n',
          '  description: Stable Diffusion XL base model (12 GB)\n',
          '  variant: normal\n',
          '  format: diffusers\n'
        ]
        file.writelines(lines)

Step 7: Starting the app

In [ ]:
#@markdown # Option 1: Starting the Web UI with ngrok
!pip install pyngrok

from pyngrok import ngrok, conf
import fileinput
import sys

Ngrok_token = "" #@param {type:"string"}
#@markdown - Add ngrok token (obtainable from https://ngrok.com)

#@markdown Only works with InvokeAI 3.0.2 and later

share=''
if Ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(9090 , pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token),
                    bind_tls=True).public_url
  print(srv)
  get_ipython().system("invokeai-web  --root /content/invokeai/")
else:
  print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the ngrok_token field.')

In [ ]:
#@markdown # Option 2: Starting the Web UI with RemoteMoe

#@markdown Wait for the line that says "Uvicorn running on http://127.0.0.1:9090 (Press CTRL+C to quit)"
#@markdown and click on the link that ends with .remote.moe

!mkdir  ~/.ssh/
!touch  ~/.ssh/known_hosts
!ssh-keyscan -t rsa remote.moe >> ~/.ssh/known_hosts
!rm /root/.ssh/id_rsa
!ssh-keygen -t rsa -b 4096 -f /root/.ssh/id_rsa -q -N ""
!invokeai-web  --root /content/invokeai/ & ssh -R 80:127.0.0.1:9090 -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe

In [ ]:
#@markdown # Option 3: Starting the Web UI with Localtunnel

%cd /content/invokeai/
!npm install -g localtunnel

# @markdown **Warning: Localtown has been down recently. If the localtunnel link does not show up in the output, go with the ngrok option.**
# @markdown
# @markdown Copy the IP address shown in the output above the line
# @markdown "your url is: https://some-random-words.loca.lt"
!wget -q -O - ipv4.icanhazip.com

#@markdown Wait for the line that says "Uvicorn running on http://127.0.0.1:9090 (Press CTRL+C to quit)"

#@markdown Click the localtunnel url and paste the IP you copied earlier to the "Endpoint IP" text field
!lt --port 9090 --local_https False & invokeai-web  --root /content/invokeai/

#@markdown If the UI shows a red dot that says 'disconnected' when hovered in the upper
#@markdown right corner and the Invoke button is disabled, change 'https' to 'http'
#@markdown in the browser's address bar and press enter.
#@markdown When the page reloads, the UI should work properly.
